# 使用情境 - 1

你現在是一個作文批改老師，你想要使用所學的Langchain知識來幫助你批改作文，建立一個作文批改的模型，輸入一篇作文，自訂輸出格式以滿足cell內題目的需求。

You should how to:

    1. define schema
    2. build output parser
    3. define imput templetes
    4. chain them together
    5. feed your agent with articles


In [6]:
import warnings
from pprint import pprint
from typing import Literal
from langchain.prompts import ChatPromptTemplate
from langchain.chains.openai_functions.base import convert_to_openai_function
from langchain.output_parsers import StructuredOutputParser, ResponseSchema, PydanticOutputParser
from langchain.output_parsers.openai_functions import PydanticOutputFunctionsParser
from langchain.pydantic_v1 import BaseModel, Field, root_validator
from langchain_setup import ChatOpenAI

In [3]:
"""
======================================================
cousre practice: 
用課程先前所學，建立一個LLM模型來批改文章，並以特定框架output

給定一篇文章 article以及其主題，須用模型給定評分
評分標準需為以下四項：切題、文筆、創意、通順

用基本串接實作
======================================================
"""

article = "愛迪生說：「友誼能增進快樂，減少痛苦」，可見朋友有多重要。朋友像一根拐杖，當我跌倒的時候，扶我起來，支撐我繼續往前走；朋友像一位溫柔的老師，當我功課不會寫時，他會很用心的教導我；朋友像一顆開心果，當我難過或傷心時，講笑話給我聽；朋友像一個太陽，當我遇到困難時，覺得人生黑暗的時候，他會溫暖我的心。猶記一年級的時候，我有一位很要好的朋友，她喜歡運動和打球，而我也很喜歡運動，下課的時候，我常常跟她一起玩。她很大方，也很有禮貌，又喜歡幫老師做事，人緣好，所以大家都選她當班長。在她身上我看到很多優點，也學到很多，有了她的陪伴，所以每天在學校都過得很開心。有一次我在操場上跌倒流血，倒在地上爬不起來，是她扶我起來，陪我到健康中心。接下來那幾天，她幫我打掃、交作業，這讓我深深體會好朋友是多麼重要。雖然我們升上中年級後沒有在同一班，但我們還是思念著對方，下課時常常聚在一起，我的心中仍將她視為最好的朋友。如果一個人沒有朋友，就找不到陪她一起玩樂和訴苦的人，這樣就會很孤單。朋友是一生中絕對不能少的，缺少了他，我可能每天心情不好而感到孤獨寂寞。有了好朋友，是一件很幸福的事，也要懂得珍惜，建立的友誼才能長久。"
title = "朋友"

# ======================================================
# Define schema,validation and postprocessing
# ======================================================
response_schemas = [
    ResponseSchema(name="切題", description="文章內容與題目的相關程度。介於 1 到 10。", type='integer'),
    ResponseSchema(name="文筆", description="文字遣詞是否精練，以及其內容深度。介於 1 到 10。", type='integer'),
    ResponseSchema(name="創意", description="文章發想內容的創意程度。介於 1 到 10。", type='integer'),
    ResponseSchema(name="通順", description="文章的通順程度。介於 1 到 10。", type='integer'),
]

structure_parser = StructuredOutputParser.from_response_schemas(response_schemas)

# ======================================================
# Define input template
# ======================================================
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一名國文教師，你現在要批改學生的作文，題目為朋友，評分標準需為以下四項：切題、文筆、創意、通順。"),
        ("human", "以下為作文：{article}"),
        ("human", "請為上述作文評分請依照以下指示回答\n{format_instructions}\n。"),
    ],
)

prompt_template2 = prompt_template.partial(format_instructions=structure_parser.get_format_instructions())

# ======================================================
# Input article and output result
# ======================================================
messages = prompt_template2.format_messages(article = article)
model = ChatOpenAI(temperature=0)
response_message = model(messages)
print(structure_parser.parse(response_message.content))

{'切題': 10, '文筆': 8, '創意': 7, '通順': 9}


In [5]:
"""
======================================================
cousre practice: 
用課程先前所學，建立一個LLM模型來批改文章，並以特定框架output

給定一篇文章 article，需用模型給出做到以下四件事：

1. 總結出此篇文章的標題
2. 依照內容給予評分，分數須介於 1~10分之間
3. 給此篇文章評語、評分依據以及可改進的地方
4. 重新改寫此篇文章，並給出新的標題

用LCEL實作
===================================================
"""

# ======================================================
# Define schema,validation and postprocessing
# ======================================================
class Article(BaseModel):
    """批改作文的老師"""

    title: str = Field(description='這篇文章的標題，請根據內容總結出最適合的標題')
    grade: int = Field(description='這篇文章的分數，請依照文章內容給分，分數介於 1~10 分之間')
    feedback: str = Field(description='這篇文章的評語，請仔細說明為何給這樣的分數以及有哪些地方可以改進')
    rewrite:str = Field(description='重新改寫這篇文章，請根據評語重新改寫文章，並且給出新的標題')

    @root_validator() 
    def validate_values(cls, values: dict) -> dict:
        if not (1 <= values['grade'] <= 10):
            warnings.warn(f"分數 {values['grade']} 不在 [1,10] 內，自動調整為 1 或 10")
            values['grade'] = max(min(1, values['grade']), 10)
        return values

# build ouput parser
output_parser = PydanticOutputFunctionsParser(pydantic_schema=Article)

# ======================================================
# Define input template
# ======================================================
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一個專門批改學生作文的老師，現在學生要請你批改作文，請滿足他們的要求。"),
        ("human", "你好，請問可以批改我的作文嗎。"),
        ("ai", "好的，請給我看你的文章。"),
        ("human", "這是我的文章 : {article}\n。"), 
    ],
)

# ======================================================
# Chain it together
# ======================================================
model = ChatOpenAI(temperature=0)
function_kwargs = convert_to_openai_function(Article)
openaifn_kwargs = {'functions': [function_kwargs], 'function_call': {'name': 'Article'}}
chain = prompt_template | model.bind(**openaifn_kwargs) | output_parser

output = chain.invoke({"article": "穿越你心中夏豔的森林\n樹葉翻飛說些什麼\n你的心溫煦敞亮\n讓人願意風塵僕僕旅行\n選一片葉寫下我的名字\n當時間順流而下\n仍有風逆行回來\n一遍一遍朗讀"})
print("輸出的資料形式是 pydantic schema:", type(output))
print({'標題': output.title, '分數': output.grade, '評語': output.feedback,'改寫':output.rewrite})

輸出的資料形式是 pydantic schema: <class '__main__.Article'>
{'標題': '穿越你心中夏豔的森林', '分數': 6, '評語': '這篇文章的詩意和意境很美，但是在表達上有些不清晰。建議在文章中加入更多具體的描述和情感表達，讓讀者更能感受到你想傳達的意思。', '改寫': '穿越你心中夏豔的森林\n樹葉翻飛，低語著秘密\n你的心溫暖而明亮\n讓人願意放下煩憂，踏上旅程\n我選了一片葉子，寫下你的名字\n當時間匆匆流逝\n仍有風逆行回來\n我一遍一遍朗讀\n感受著你的存在。'}


# 使用情境 - 2

請同學利用情境分支 - RunnableBranch來處理兩種不同種類的問題，並建立兩個語言模型來回答：
    
  - 第一個種類的為問題內包含「景點」的問句，建立一個chain來回答使用者的問題並推薦景點
  - 第二個種類的為問題內包含「美食」的問句，建立另一個chain來回答使用者的問題並推薦食物
        
    
  - 範例如下：
    
            輸入：台灣有什麼好吃的美食嗎
            輸出：
                Scenario Branch Output:
                ('臭豆腐', '臭豆腐是台灣非常受歡迎的特色小吃之一。它是由豆腐經過發酵製成，因此帶有一種獨特的臭味。雖然聞起來可能讓人有些驚訝，但臭豆腐的味道卻是令人著迷的。它通常會先炸至外酥內軟，再搭配醬料，如醬油、辣椒醬等，增添風味。臭豆腐酥脆外皮與軟嫩內餡的組合，讓人一試難忘。')
                
            輸入：台灣有什麼好玩的景點嗎
            輸出：
               Scenario Branch Output:
                {'景點': '九份老街', '介紹': '九份老街是一個保存完整的日本式街道，位於台灣基隆市內門里，是一個歷史悠久的地區。這條街道上有許多傳統的小店，販賣著各式各樣的特色商品和美食。遊客可以在這裡品嚐到台灣傳統的小吃，如芋圓、鳳梨酥和茶葉蛋，也可以購買紀念品和手工藝品。此外，九份老街的風景也非常迷人，遊客可以欣賞到壯麗的山景和海景，還可以漫步在狹窄的巷弄中，感受悠閒的氛圍。'}
    

In [33]:
from langchain.prompts import ChatPromptTemplate
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain_setup import ChatOpenAI
from langchain.schema.runnable import RunnableLambda, RunnableMap, RunnableBranch, RunnablePassthrough

# Scenario: Developing a Chatbot Pipeline

# Components Initialization
# Prompt
prompt = ChatPromptTemplate.from_messages(
    [
    ("system", "你是一位智能導遊機器人。請根據輸入的疑問滿足旅客的要求。"),
    ("human","請告訴我{place}有什麼好玩的，並推薦當地的一個景點並向我介紹\n\n{format_instructions}\n\n"),
],
)

# Model
model = ChatOpenAI()

# Output Parser
response_schemas = [
    ResponseSchema(name="景點", description="推薦的旅遊景點"),
    ResponseSchema(name="介紹", description="景點的簡要介紹"),
]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

# Exercise:
prompt = prompt.partial(format_instructions=output_parser.get_format_instructions())

# 1. Constructing the Basic Chain - Prompt | Model | Output Parser
attraction_chain = prompt | model | output_parser

# 2. Invoking the Basic Chain
attraction_output = attraction_chain.invoke({"place":"東京"})
print("\nBasic Chain Output:")
print(attraction_output)


Basic Chain Output:
{'景點': '東京迪士尼樂園', '介紹': '東京迪士尼樂園是日本最著名的主題樂園之一，也是世界上最受歡迎的遊樂園之一。樂園內有兩個主題公園：東京迪士尼樂園和東京迪士尼海洋。東京迪士尼樂園擁有許多經典的迪士尼遊樂設施，如太空山、小美人魚的冒險和白雪公主的冒險故事。遊客可以在這裡與迪士尼角色合影、觀賞精彩的表演和遊覽迷人的花園。無論是大人還是小孩，都能在東京迪士尼樂園度過一個令人難忘的時光。'}


In [38]:
# Creating RunnableLambda instances for the functions

prompt = ChatPromptTemplate.from_messages(
    [
    ("system", "你是一位智能導遊機器人。請根據輸入的疑問滿足旅客的要求。"),
    ("human", "請告訴我{place}有什麼特色美食，簡要的介紹一個特色美食給我\n\n{format_instructions}\n\n"),
],
)

# Model
model = ChatOpenAI()

# Output Parser
response_schemas = [
    ResponseSchema(name="美食", description="美食名稱"),
    ResponseSchema(name="介紹", description="美食的簡要介紹"),
]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

# Exercise:
prompt = prompt.partial(format_instructions=output_parser.get_format_instructions())

# 3. Combining Runnable Functions using RunnableLambda
food_chain = prompt | model | output_parser

# 4. Invoking the Advanced Chain
food_output = food_chain.invoke({"place":"東京"})
print("\nFood Chain Output:")
print(food_output)

# 5. Handling Multiple Scenarios - RunnableBranch

def route(question: str):
    if "景點" in question:
        return attraction_chain.invoke({"place": question})
    elif "美食" in question:
        return food_chain.invoke({"place": question})
    else:
        return "很抱歉，我不太明白您的需求。"

route_runnable = RunnableLambda(route)

# 6. Invoking the Scenario Branch
scenario_input = "台灣有什麼好玩的景點嗎?"
scenario_output = route_runnable.invoke(scenario_input)
print("\nScenario Branch Output:")
print(scenario_output)


Food Chain Output:
{'美食': '拉麵', '介紹': '拉麵是一道源自中國的日本特色美食，它由彈性的麵條和濃郁的湯頭組成。這種湯頭通常是以豬骨、雞骨或魚骨熬煮而成的，味道濃郁鮮美。拉麵的配料有很多種類，例如叉燒肉、海苔、蔥花等，可以根據個人喜好選擇。在東京，你可以找到各種口味的拉麵，包括豚骨拉麵、醬油拉麵、味噌拉麵等。無論你喜歡濃郁的湯頭還是清淡的口味，東京的拉麵一定能滿足你的味蕾。'}

Scenario Branch Output:
{'景點': '九份老街', '介紹': '九份是一個位於台灣基隆市的小漁村，以其古老的街道和懷舊的風格而聞名。九份老街是九份最熱門的景點之一，這條狹窄的街道上擺滿了各種小吃攤位、紀念品店和茶館。遊客可以在這裡品嚐到台灣特色小吃，如芋圓、花生捲和鳳梨酥，同時還可以欣賞到壯麗的海景和山景。九份老街也是電影《悲情城市》的取景地，吸引了眾多電影愛好者前來觀光。'}
